# 2.2 数据预处理

### 2.2.1 读取数据集  
首先创建一个人工数据集，并存储在CSV（逗号分隔值）文件

In [2]:
import os

os.makedirs(os.path.join('.', 'data'), exist_ok=True)
data_file = os.path.join('.', 'data', 'house_tiny.csv')
with open(data_file, 'w') as f:
    f.write('NumRooms,Alley,Price\n')  # 列名
    f.write('NA,Pave,127500\n')  # 每行表示一个数据样本
    f.write('2,NA,106000\n')
    f.write('4,NA,178100\n')
    f.write('NA,NA,140000\n')

In [3]:
# 从创建的CSV文件中加载原始数据集
# 导入pandas包并调用read_csv函数
import pandas as pd

data = pd.read_csv(data_file)
print(data)

   NumRooms Alley   Price
0       NaN  Pave  127500
1       2.0   NaN  106000
2       4.0   NaN  178100
3       NaN   NaN  140000


### 2.2.2 处理缺失值  
“NaN”项代表缺失值  
典型的方法包括插值法和删除法

In [20]:
# 在这里，我们将考虑插值法
# 通过位置索引iloc，我们将data分成inputs和outputs
# 对于inputs中缺少的数值，我们用同一列的均值替换“NaN”项
inputs, outputs = data.iloc[:, 0:2], data.iloc[:, 2]
# print(inputs.iloc[:, 1])
inputs.iloc[:, 0] = inputs.fillna(inputs.iloc[:, 0].mean())
print(inputs)

   NumRooms Alley
0       3.0  Pave
1       2.0   NaN
2       4.0   NaN
3       3.0   NaN


In [21]:
# 对于inputs中的类别值或离散值，我们将“NaN”视为一个类别。
# 由于“巷子类型”（“Alley”）列只接受两种类型的类别值“Pave”和“NaN”
# 自动将此列转换为两列“Alley_Pave”和“Alley_nan”。
inputs = pd.get_dummies(inputs, dummy_na=True)
print(inputs)

   NumRooms  Alley_Pave  Alley_nan
0       3.0        True      False
1       2.0       False       True
2       4.0       False       True
3       3.0       False       True


### 2.2.3 转换为张量格式

In [22]:
# 现在inputs和outputs中的所有条目都是数值类型
# 可以转换为张量格式
import torch

X = torch.tensor(inputs.to_numpy(dtype=float))
y = torch.tensor(outputs.to_numpy(dtype=float))
X, y

(tensor([[3., 1., 0.],
         [2., 0., 1.],
         [4., 0., 1.],
         [3., 0., 1.]], dtype=torch.float64),
 tensor([127500., 106000., 178100., 140000.], dtype=torch.float64))

### 2.2.4 小结  

1. pandas软件包是Python中常用的数据分析工具中，pandas可以与张量兼容
2. 用pandas处理缺失的数据时，我们可根据情况选择用插值法和删除法。

### 2.2.5 练习
创建包含更多行和列的原始数据集

In [27]:
os.makedirs(os.path.join('.', 'data'), exist_ok=True)
data_file = os.path.join('.', 'data', 'house_tiny_more.csv')
with open(data_file, 'w') as f:
    f.write('NumRooms,Alley,Price\n')  # 列名
    f.write('NA,Pave,127500\n')  # 每行表示一个数据样本
    f.write('2,NA,106000\n')
    f.write('4,NA,178100\n')
    f.write('NA,NA,140000\n')
    f.write('NA,Pave,127500\n')  # 每行表示一个数据样本
    f.write('2,NA,106000\n')
    f.write('4,NA,178100\n')
    f.write('NA,NA,140000\n')


In [28]:
# 1. 删除缺失值最多的列
import pandas as pd

# 读取数据
data = pd.read_csv(data_file)
print(data)

# 计算每个列的缺失值数量
missing_values = data.isnull().sum()

# 找到缺失值数量最多的列
max_missing_values_column = missing_values.idxmax()

# 删除缺失值最多的列
data = data.drop(max_missing_values_column, axis=1)

# 打印处理后的数据
print(data)

   NumRooms Alley   Price
0       NaN  Pave  127500
1       2.0   NaN  106000
2       4.0   NaN  178100
3       NaN   NaN  140000
4       NaN  Pave  127500
5       2.0   NaN  106000
6       4.0   NaN  178100
7       NaN   NaN  140000
   NumRooms   Price
0       NaN  127500
1       2.0  106000
2       4.0  178100
3       NaN  140000
4       NaN  127500
5       2.0  106000
6       4.0  178100
7       NaN  140000


In [29]:
# 2. 将预处理后的数据集转换为张量格式。
import torch
inputs, outputs = data.iloc[:, 0:1], data.iloc[:, -1]
inputs = torch.tensor(inputs.to_numpy(dtype=float))
outputs = torch.tensor(outputs.to_numpy(dtype=float))
inputs, outputs

(tensor([[nan],
         [2.],
         [4.],
         [nan],
         [nan],
         [2.],
         [4.],
         [nan]], dtype=torch.float64),
 tensor([127500., 106000., 178100., 140000., 127500., 106000., 178100., 140000.],
        dtype=torch.float64))